In [1]:
# import and dataset overview
import pandas as pd
df = pd.read_csv("/content/dataset/persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [2]:
def check_df(dataframe, head=5):
  print("########## SHAPE ##########")
  print(dataframe.shape)
  print("########## TYPES ##########")
  print(dataframe.dtypes)
  print("########## HEAD ##########")
  print(dataframe.head(head))
  print("########## TAIL ##########")
  print(dataframe.tail(head))
  print("########## NA ##########")
  print(dataframe.isnull().sum())
  print("########## QUANTILES ##########")
  print(dataframe.describe([0, 0.05, 0.50, 0.95, 1]).T)

check_df(df)

########## SHAPE ##########
(5000, 5)
########## TYPES ##########
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
########## HEAD ##########
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
########## TAIL ##########
      PRICE   SOURCE     SEX COUNTRY  AGE
4995     29  android  female     bra   31
4996     29  android  female     bra   31
4997     29  android  female     bra   31
4998     39  android  female     bra   31
4999     29  android  female     bra   31
########## NA ##########
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
########## QUANTILES ##########
        count     mean        std   min    0%    5%   50%   95%  100%   max
PRICE  5000.0  34.1320  12.464897   9.0   9.0  19.0  39.0  49.0  59.0  59.0
AGE    5000.0  

In [3]:
# How many unique SOURCE are there? What are their frequencies?
df["SOURCE"].nunique()

2

In [4]:
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [5]:
# How many unique PRICE are there?
df["PRICE"].nunique()

6

In [6]:
# How many sales were made from which PRICE?
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [7]:
# How many sales from which country?
df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [8]:
# df.groupby("COUNTRY")["PRICE"].count()
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


In [9]:
# How much was earned in total from sales by country?
# df.groupby("COUNTRY")["PRICE"].sum()
# df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="sum")
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [10]:
# What are the sales numbers according to SOURCE types?
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [11]:
# What are the PRICE averages by country?
df.groupby(by=['COUNTRY']).agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [12]:
# What are the PRICE averages by SOURCE?
df.groupby(['SOURCE']).agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [13]:
# What is the average PRICE in the COUNTRY-RESOURCE refraction ?
df.groupby(["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [14]:
# What are the average earnings in COUNTRY, SOURCE, SEX, AGE refraction ?
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head(10)

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
                       20   31.000000
                       21   37.292683
                       22   34.945946
                       23   31.941176
                       24   36.500000

In [15]:
# Let's sort the output by PRICE in descending order.
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)

In [16]:
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [17]:
# Let's turn the names in the index into variable names. (All variables except PRICE in the above output are index names.)
agg_df.reset_index(inplace=True)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [18]:
# Let's convert the AGE variable to a categorical variable and add it to agg_df.
age_labels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())]
bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

In [19]:
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins, labels=age_labels)
agg_df.head(10)

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40
5,deu,android,female,51,49.0,41_66
6,deu,ios,male,20,49.0,19_23
7,usa,android,female,47,49.0,41_66
8,fra,ios,male,20,49.0,19_23
9,usa,ios,male,59,46.5,41_66


In [20]:
# Let's define new level based customers and add them as variables to the dataset.
agg_df["CUSTOMERS_LEVEL_BASED"] = ['_'.join(i).upper() for i in agg_df.drop(["AGE", "PRICE"], axis=1).values]
agg_df.head(10)

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,CUSTOMERS_LEVEL_BASED
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40
5,deu,android,female,51,49.0,41_66,DEU_ANDROID_FEMALE_41_66
6,deu,ios,male,20,49.0,19_23,DEU_IOS_MALE_19_23
7,usa,android,female,47,49.0,41_66,USA_ANDROID_FEMALE_41_66
8,fra,ios,male,20,49.0,19_23,FRA_IOS_MALE_19_23
9,usa,ios,male,59,46.5,41_66,USA_IOS_MALE_41_66


In [21]:
# Let's edit it so that only the necessary variables remain.
agg_df = agg_df[["CUSTOMERS_LEVEL_BASED", "PRICE"]]
agg_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE
0,BRA_ANDROID_MALE_41_66,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


In [22]:
# In order not to have more than one from the same segment, let's group by segments and get the average price, 
# then convert the customer_level_based in the index into a variable.
agg_df = agg_df.groupby("CUSTOMERS_LEVEL_BASED").agg({"PRICE": "mean"})
agg_df = agg_df.reset_index()
agg_df.head(10)

,CUSTOMERS_LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179
5,BRA_ANDROID_MALE_0_18,34.805861
6,BRA_ANDROID_MALE_19_23,31.673243
7,BRA_ANDROID_MALE_24_30,33.413919
8,BRA_ANDROID_MALE_31_40,34.327381
9,BRA_ANDROID_MALE_41_66,40.041667


In [23]:
# Let's segment the new customers (USA_ANDROID_MALE_0_18).
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["A", "B", "C", "D"])
agg_df.head(10)

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,C
1,BRA_ANDROID_FEMALE_19_23,34.077340,B
2,BRA_ANDROID_FEMALE_24_30,33.863946,B
3,BRA_ANDROID_FEMALE_31_40,34.898326,C
4,BRA_ANDROID_FEMALE_41_66,36.737179,D
5,BRA_ANDROID_MALE_0_18,34.805861,C
6,BRA_ANDROID_MALE_19_23,31.673243,A
7,BRA_ANDROID_MALE_24_30,33.413919,B
8,BRA_ANDROID_MALE_31_40,34.327381,C
9,BRA_ANDROID_MALE_41_66,40.041667,D


In [24]:
# Let's examine the descriptive statistics of the segments (mean, maximum value, total, median).
agg_df.groupby("SEGMENT").agg({"PRICE": ["mean","max","sum", "median"]})

PRICE                                   
              mean        max          sum     median
SEGMENT                                              
A        29.206780  32.333333   817.789833  30.956140
B        33.509674  34.077340   904.761209  33.627634
C        34.999645  36.000000   944.990411  34.900410
D        38.691234  45.428571  1044.663328  37.685764

In [25]:
# Let's take a few examples to classify new customers and estimate how much revenue they can generate.
# What segment does a 33-year-old Turkish woman using ANDROID belong to and how much income is expected to earn on average?
new_customer = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == new_customer]

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,D


In [26]:
# What segment does a 35-year-old French woman using IOS belong to and how much income is expected to earn on average?
new_customer = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == new_customer]

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,B
